In [1]:
# import matplotlib.pyplot as plt
# from scipy.ndimage import rotate
from src.utils_for_estimate import (get_points,
                                    get_image_center,
                                    find_heights_and_shadows,
                                    extreme_points,
                                    extra_rotate_angle,
                                    find_horizontal_area,
                                    )
from src.shadow_height_estimator import (
                                        attach_heights, 
                                        rotate_points,
                                        estimate_buildings_height,
                                        attach_heights,
                                        buildings_info
                                        
                                        )
# from src.visualization import (draw_masks,
#                                     draw_random_masks,
#                                     draw_sample_building_shadow,
#                                     )
from src.annotations.annotation import anns_and_images
import numpy as np

In [2]:
dataset_dir = '../3d_tools_dataset/'
annot_file = dataset_dir + 'annotations/instances_default.json'
images_directory = dataset_dir + 'image_png/'
image_directory = dataset_dir + 'image_png/test_1.png'

In [3]:
SCALE = np.float16(0.1)
SUN_AZIMUTH_ANGLE = np.uint8(100)
SUN_ELEVATION_ANGLE = np.uint8(30)
ROTATE_ANGLE = SUN_AZIMUTH_ANGLE + 90.0
THRESHOLD = np.uint8(5)
BROWN = (np.uint8(128), np.uint8(64), np.uint8(48))
BLUE = (np.uint8(0), np.uint8(0), np.uint8(255))
COLORS = np.array([BROWN, BLUE])

In [4]:
image, anns = anns_and_images(coco_annotation_file=annot_file, images_directory=images_directory)

shadows_points = get_points(anns)[0]
building_points = get_points(anns)[1]

center = get_image_center(image_path=image_directory)
rotated_points_of_shadows = rotate_points(points=shadows_points, center=center, rotate_angle=ROTATE_ANGLE)
rotated_points_of_buildings = rotate_points(points=building_points, center=center, rotate_angle=ROTATE_ANGLE)

extra_angles = {}
list_of_areas = []
for iteration, points_of_shadow in rotated_points_of_shadows.items():
    area = find_horizontal_area(points_of_shadow, THRESHOLD)
    extr_poi = extreme_points(points_of_shadow, area)
    extra_angles[iteration] = extra_rotate_angle(extr_poi)
    
shadows_heigth = estimate_buildings_height(rotated_points_of_shadows, SCALE, SUN_ELEVATION_ANGLE)

find = find_heights_and_shadows(rotated_points_of_shadows,
                                rotated_points_of_buildings,
                                shadows_heigth,
                                THRESHOLD
                                )

attach = attach_heights(shadows_heigth, find)

# print(extra_angles)
# building_heights_shadows {'building_heights': {3: [27.64], 4: [7.25], 5: [8.66], 6: [22.03], 8: [13.61], 9: [9.59], 12: [10.08], 13: [7.9]},
# 'building_shadow': {3: [2], 4: [10], 5: [15], 6: [1], 8: [7], 9: [0], 12: [11], 13: [14]}}

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
{9: {'anns': {'id': 10, 'image_id': 1, 'category_id': 1, 'segmentation': [[4136.33, 1680.84, 4474.11, 1781.74, 4407.38, 2015.64, 4064.78, 1910.13, 4125.71, 1711.3]], 'area': 86062.0, 'bbox': [4064.78, 1680.84, 409.33, 334.8], 'iscrowd': 0, 'attributes': {'occluded': False}}, 'heights': 6.84}, 3: {'anns': {'id': 4, 'image_id': 1, 'category_id': 1, 'segmentation': [[4192.8, 999.35, 4384.2, 1057.68, 4389.92, 1025.13, 4414.66, 875.49, 4406.42, 874.01, 4411.62, 850.23, 4424.47, 847.63, 4431.68, 803.79, 4427.07, 796.07, 4420.91, 794.51, 4440.15, 680.08, 4445.87, 681.64, 4463.55, 570.64, 4461.77, 561.2, 4457.09, 556.67, 4472.99, 466.39, 4519.28, 472.86, 4530.05, 404.95, 4530.05, 377.01, 4448.4, 363.78, 4449.44, 353.01, 4375.95, 341.57, 4374.91, 353.01, 4289.99, 340.16, 4268.51, 487.72, 4272.45, 489.2, 4272.82, 483.48, 4276.39, 483.78, 4251.13, 629.04, 4243.7, 628.0, 4229.51, 731.8, 4237.68, 730.76, 4227.5, 803.

In [5]:
hyperparameters = {'anns': anns,
                'sun_azimuth_angle': SUN_AZIMUTH_ANGLE,
                'scale': SCALE,
                'sun_elevation_angle': SUN_ELEVATION_ANGLE,
                'threshold': THRESHOLD,
                'img_center': center
                }

In [6]:
build_info = buildings_info(hyperparameters)
print(build_info)

{9: {'anns': {'id': 10, 'image_id': 1, 'category_id': 1, 'segmentation': [[4136.33, 1680.84, 4474.11, 1781.74, 4407.38, 2015.64, 4064.78, 1910.13, 4125.71, 1711.3]], 'area': 86062.0, 'bbox': [4064.78, 1680.84, 409.33, 334.8], 'iscrowd': 0, 'attributes': {'occluded': False}}, 'heights': 6.84}, 3: {'anns': {'id': 4, 'image_id': 1, 'category_id': 1, 'segmentation': [[4192.8, 999.35, 4384.2, 1057.68, 4389.92, 1025.13, 4414.66, 875.49, 4406.42, 874.01, 4411.62, 850.23, 4424.47, 847.63, 4431.68, 803.79, 4427.07, 796.07, 4420.91, 794.51, 4440.15, 680.08, 4445.87, 681.64, 4463.55, 570.64, 4461.77, 561.2, 4457.09, 556.67, 4472.99, 466.39, 4519.28, 472.86, 4530.05, 404.95, 4530.05, 377.01, 4448.4, 363.78, 4449.44, 353.01, 4375.95, 341.57, 4374.91, 353.01, 4289.99, 340.16, 4268.51, 487.72, 4272.45, 489.2, 4272.82, 483.48, 4276.39, 483.78, 4251.13, 629.04, 4243.7, 628.0, 4229.51, 731.8, 4237.68, 730.76, 4227.5, 803.94, 4219.33, 799.71, 4204.39, 897.78, 4194.59, 946.75, 4193.77, 963.39]], 'area': 1